# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [17]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_data = "../output_data/cities.csv"

cities_df = pd.read_csv(weather_data)
cities_df

,City,Lat,Lng,Max Temp,Humidity,Clouds,Wind Speed,Country,Date
0,Punta Arenas,-53.1500,-70.9167,10.73,61,75,2.37,CL,1620926416
1,Saint-Philippe,-21.3585,55.7679,25.03,78,35,4.39,RE,1620926534
2,Amapá,1.0000,-52.0000,23.79,98,100,0.34,BR,1620926535
3,Port Elizabeth,-33.9180,25.5701,16.24,88,0,2.06,ZA,1620926535
4,Baiyin,36.5583,104.2081,47.35,54,32,5.48,CN,1620926535
...,...,...,...,...,...,...,...,...,...
559,Vila Franca do Campo,37.7167,-25.4333,20.51,66,65,3.98,PT,1620926688
560,Horn Lake,34.9554,-90.0348,21.62,32,1,4.92,US,1620926408
561,Puerto Quijarro,-17.7833,-57.7667,29.35,39,99,1.15,BO,1620926688
562,Altea,38.5989,-0.0514,28.04,91,27,5.78,ES,1620926688


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key = g_key)

# Define locations
locations = cities_df[["Lat", "Lng"]]

# Define humidity
#humidity = output_df["Humidity"].astype(float)
humidity = cities_df["Humidity"]

#fig = gmaps.figure()
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)
max_intensity = np.max(humidity)

# Create heat layer


heatmap_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=100, point_radius=3)

fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [15]:
ideal_cities_df = cities_df.loc[(cities_df["Wind Speed"] <= 10) & (cities_df["Clouds"] == 0)]
                                
      ## COMMENTED OUT BECAUSE NONE MATCH GIVEN CRITERIA       & cities_df["Max Temp"] >= 70) & (cities_df["Max Temp"] <= 80)]

ideal_cities_df.head()

,City,Lat,Lng,Max Temp,Humidity,Clouds,Wind Speed,Country,Date
3,Port Elizabeth,-33.9180,25.5701,16.24,88,0,2.06,ZA,1620926535
14,Bredasdorp,-34.5322,20.0403,15.85,82,0,2.69,ZA,1620926538
28,Otse,-25.0167,25.7333,31.55,19,0,0.96,BW,1620926542
53,Mar del Plata,-38.0023,-57.5575,20.00,51,0,2.24,AR,1620926432
58,Katherine,-14.4667,132.2667,19.18,42,0,2.76,AU,1620926406


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [18]:
#Import API key
from api_keys import g_key

# Configure gmaps; addressing issues with g_key
gmaps.configure(api_key = g_key)

# Create hotel file
hotel_df = ideal_cities_df.reset_index(drop=True)

# Add Hotel Name column
hotel_df["Hotel Name"] = ""

#Define params
params = {
    "radius": 5000,
    "types" : "hotel",
    "keyword" : "hotels",
    "key" : g_key
}
    
# Loop and use lat/lng to identify hotels
for index, row in hotel_df.iterrows():
    
    # Define latitudes and longitudes
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Maintain parameters with various locations
    params["location"] = f"{lat},{lng}"
    
    # Define url
    main_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # Capture hotel name
    hotel_name = requests.get(main_url, params = params)
    
    # Convert to json file
    hotel_name = hotel_name.json()
    print(json.dumps(hotel_name, indent=4, sort_keys=True))
    
    # Include option to skip missing data
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_name["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing result.... Skipping.")
hotel_df

{
    "html_attributions": [],
    "next_page_token": "ATtYBwKXzP4SimT_CEgpvrhaK-K1N2_lIX6nVkv_dIWAGD3GIjhGDqf1xnn9jxtvCFY7hijBQ7cnUj4pQP-YQM9-Y0-nCC_URUJu9pXCe2sCwAv4iLhva1wU4Wuq05V3lbkGPbml-H6-TUwWeCrCnhT6cPp1bJZqpTMbdPdP2qfQSshdddwqz6oe5TMIvSXkhfb77CnlXcg8739bb-jWhP7tVhtpVH1jU48ejRqL7IWlPfz6DZ0NOKzwuaukH_27WFmIhBkf_tLo5W5fBE2lbmBNUYo3WFQt6A4tFN12LQYMQ5WRsBMqfw4G5E1lCCL5JdXgid6nqMYhAkk-jhrzLUDtzHazgXSY73vevC-LzJQN7IcfV8zlSF7HqQReyNNBsAqXvzEiKpMK1FLKMzlzCwV5H33fyeyF9DUxMQAZdRPlpXEbGu6xVvil0MhR_gyvrdRu",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -33.94592,
                    "lng": 25.593
                },
                "viewport": {
                    "northeast": {
                        "lat": -33.94451792010727,
                        "lng": 25.59427402989272
                    },
                    "southwest": {
                        "lat": -33.947217579892

KeyboardInterrupt: 

In [19]:
hotel_df
hotel_df.dropna()
hotel_df.head(10)

,City,Lat,Lng,Max Temp,Humidity,Clouds,Wind Speed,Country,Date,Hotel Name
0,Port Elizabeth,-33.9180,25.5701,16.24,88,0,2.06,ZA,1620926535,Island Way Villa
1,Bredasdorp,-34.5322,20.0403,15.85,82,0,2.69,ZA,1620926538,Firlane House
2,Otse,-25.0167,25.7333,31.55,19,0,0.96,BW,1620926542,
3,Mar del Plata,-38.0023,-57.5575,20.00,51,0,2.24,AR,1620926432,Hotel Las Rocas
4,Katherine,-14.4667,132.2667,19.18,42,0,2.76,AU,1620926406,ibis Styles Katherine
5,Dunedin,-45.8742,170.5036,8.22,66,0,4.92,NZ,1620926433,Distinction Dunedin Hotel
6,São Filipe,14.8961,-24.4956,25.19,70,0,2.16,CV,1620926484,Hotel Xaguate
7,Port Alfred,-33.5906,26.8910,17.08,81,0,2.94,ZA,1620926561,Oceana Beach Wildlife Reserve
8,Ormara,25.2088,64.6357,28.20,83,0,7.02,PK,1620926568,Alqadri hotel zero point ormara
9,kimparana,12.8347,-4.9244,42.44,9,0,3.60,ML,1620926571,


In [20]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [21]:
# Add marker layer ontop of heat map
info_box_content = hotel_info
markers = gmaps.marker_layer(locations, hotel_info)
marker_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)


# Display figure
fig

Figure(layout=FigureLayout(height='420px'))